<a href="https://colab.research.google.com/github/romerocruzsa/cp-anemia-detection/blob/capstone-benchmark/notebooks/capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchinfo
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
# Import necessary libraries for file handling, data manipulation, and visualization
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from timm import create_model

# Import libraries for working with images and transformations
from PIL import Image
import cv2 as cv

# Import PyTorch modules for model building, data handling, and evaluation
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import torchvision.models.quantization as quant_models
from torch.utils.data import Dataset, DataLoader, Subset
from torchinfo import summary

# Import libraries for machine learning metrics and model evaluation
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, mean_squared_error, mean_absolute_error, r2_score
import torchmetrics
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
import gc

# Set the seed.
seed = 42
torch.manual_seed(seed)

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
# data_dir="/workspace/cp-anemia-detection/data/cp-anemia/"
# weights_dir="/workspace/cp-anemia-detection/data/notebooks/weights/"

data_dir = "/content/drive/MyDrive/CAWT_Sebastian_202425/CP-AnemiC/"
weights_dir = "/content/drive/MyDrive/CAWT_Sebastian_202425/Weights/"
anemic_dir=data_dir+"/Anemic/"
non_anemic_dir=data_dir+"/Non-anemic/"
signature = "02042024"

In [5]:
data_sheet_path = data_dir+"Anemia_Data_Collection_Sheet.csv"
data_sheet = pd.read_csv(data_sheet_path)
display(data_sheet)

,IMAGE_ID,HB_LEVEL,Severity,Age(Months),GENDER,REMARK,HOSPITAL,CITY/TOWN,MUNICIPALITY/DISTRICT,REGION,COUNTRY
0,Image_001,9.80,Moderate,6,Female,Anemic,Nkawie-Toase Government Hospital,Nkawie-Toase,Atwima Nwabiagya South,Ashanti,Ghana
1,Image_002,9.90,Moderate,24,Male,Anemic,Ejusu Government Hospital,Ejusu,Ejusu Municipality,Ashanti,Ghana
2,Image_003,11.10,Non-Anemic,24,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
3,Image_004,12.50,Non-Anemic,12,Male,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
4,Image_005,9.90,Moderate,24,Male,Anemic,Sunyani Municipal Hospital,Sunyani,Sunyani Municipality,Bono,Ghana
...,...,...,...,...,...,...,...,...,...,...,...
705,Image_706,12.80,Non-Anemic,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana
706,Image_707,11.47,Non-Anemic,48,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
707,Image_708,11.60,Non-Anemic,60,Male,Non-anemic,Komfo Anokye Teaching Hospital,Kumasi,Kumasi Metropolitan,Ashanti,Ghana
708,Image_709,12.10,Non-Anemic,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana


In [6]:
# Mapping diagnosis to severity
severity_mapping = {
    "Non-Anemic": 0,
    "Mild": 1,
    "Moderate": 2,
    "Severe": 3,
}

data_sheet['Severity'] = data_sheet['Severity'].map(severity_mapping)
display(data_sheet)

,IMAGE_ID,HB_LEVEL,Severity,Age(Months),GENDER,REMARK,HOSPITAL,CITY/TOWN,MUNICIPALITY/DISTRICT,REGION,COUNTRY
0,Image_001,9.80,2,6,Female,Anemic,Nkawie-Toase Government Hospital,Nkawie-Toase,Atwima Nwabiagya South,Ashanti,Ghana
1,Image_002,9.90,2,24,Male,Anemic,Ejusu Government Hospital,Ejusu,Ejusu Municipality,Ashanti,Ghana
2,Image_003,11.10,0,24,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
3,Image_004,12.50,0,12,Male,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
4,Image_005,9.90,2,24,Male,Anemic,Sunyani Municipal Hospital,Sunyani,Sunyani Municipality,Bono,Ghana
...,...,...,...,...,...,...,...,...,...,...,...
705,Image_706,12.80,0,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana
706,Image_707,11.47,0,48,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
707,Image_708,11.60,0,60,Male,Non-anemic,Komfo Anokye Teaching Hospital,Kumasi,Kumasi Metropolitan,Ashanti,Ghana
708,Image_709,12.10,0,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana


In [37]:
# Define data augmentations or transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=np.random.rand()),
    transforms.RandomVerticalFlip(p=np.random.rand()),
    transforms.RandomRotation(degrees=np.random.randint(0, 360)),
    transforms.RandomAffine(degrees=np.random.randint(0, 360)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom dataset class
class CPAnemiCDataset(Dataset):
    def __init__(self, dir, df, transform=None):
        self.dir = dir
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = row['IMAGE_ID']
        img_folder = row['REMARK']
        img_path = os.path.join(self.dir, img_folder, img_id + ".png")
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        multiclass_label = torch.tensor(row['Severity'])
        hb_level = torch.tensor(row['HB_LEVEL'])

        return img, multiclass_label, hb_level

    # Load the dataset
image_dataset = CPAnemiCDataset(data_dir, data_sheet, transform=transform)
train_dataset, test_dataset = train_test_split(image_dataset, test_size=0.20, shuffle=True)

print(f"Image Dataset Size (All): {len(image_dataset)}, \
        Train Size: {len(train_dataset)}, \
        Test Size: {len(test_dataset)}")

BATCH_SIZE = 32
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

Image Dataset Size (All): 710,         Train Size: 568,         Test Size: 142


In [38]:
# Default device
device = torch.device('cpu')

# Check for CUDA availability
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print("CUDA is not available, using CPU.")

print(f"Selected device: {device}")

Selected device: cuda


In [39]:
def get_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    model_size = "Model Size: %.2f MB" %(os.path.getsize("tmp.pt")/1e6)
    os.remove('tmp.pt')
    return model_size

# Static Weighting Function. Set eta_class to desired importance (Classification > .5, Regression < .5, Equal == .5)
def sw_loss(loss_class, loss_reg, eta_class=0.5):
    eta_reg = 1 - eta_class
    total_loss = (eta_class * loss_class) + (eta_reg * loss_reg)
    return total_loss

In [40]:
def train(dataloader, model, loss_fn_class, loss_fn_reg, optimizer):
    model.train()
    total_loss = 0
    total_ce_loss = 0
    total_mse_loss = 0
    correct = 0
    total_samples = 0
    total_mae = 0

    for _, (img, multiclass, hb_level) in enumerate(dataloader):
        img = img.to(device)
        multiclass = multiclass.to(device).long()  # Ensure correct type for CrossEntropyLoss
        hb_level = hb_level.to(device).unsqueeze(1).float()

        optimizer.zero_grad()

        # Forward pass
        class_pred, reg_pred = model(img)

        # Compute losses
        ce_loss = loss_fn_class(class_pred, multiclass)  # CrossEntropy for classification
        mse_loss = loss_fn_reg(reg_pred, hb_level)  # MSE for regression
        loss = sw_loss(ce_loss, mse_loss, 0.7)  # Weighted loss

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Track total losses
        total_loss += loss.item()
        total_ce_loss += ce_loss.item()
        total_mse_loss += mse_loss.item()

        # Compute classification accuracy
        class_probs = F.softmax(class_pred, dim=1)
        highest_prob_class = torch.argmax(class_probs, dim=1)
        correct += (highest_prob_class == multiclass).sum().item()
        total_samples += multiclass.size(0)

        # Compute regression MAE
        total_mae += torch.abs(reg_pred - hb_level).sum().item()

    avg_loss = total_loss / len(dataloader)
    avg_ce_loss = total_ce_loss / len(dataloader)
    avg_mse_loss = total_mse_loss / len(dataloader)
    classification_accuracy = correct / total_samples
    regression_mae = total_mae / total_samples

    return avg_loss, classification_accuracy, regression_mae, avg_ce_loss, avg_mse_loss

In [41]:
def eval(dataloader, model, loss_fn_class, loss_fn_reg):
    model.eval()
    total_loss = 0
    total_ce_loss = 0
    total_mse_loss = 0
    correct = 0
    total_samples = 0
    total_mae = 0

    torch.cuda.empty_cache()
    gc.collect()

    with torch.no_grad():
        for _, (img, multiclass, hb_level) in enumerate(dataloader):
            img = img.to(device)
            multiclass = multiclass.to(device).long()  # Ensure correct type for CrossEntropyLoss
            hb_level = hb_level.to(device).unsqueeze(1).float()

            # Forward pass
            class_pred, reg_pred = model(img)

            # Compute losses
            ce_loss = loss_fn_class(class_pred, multiclass)  # CrossEntropy for classification
            mse_loss = loss_fn_reg(reg_pred, hb_level)  # MSE for regression
            loss = sw_loss(ce_loss, mse_loss, 0.7)  # Weighted loss

            # Track total losses
            total_loss += loss.item()
            total_ce_loss += ce_loss.item()
            total_mse_loss += mse_loss.item()

            # Compute classification accuracy
            class_probs = F.softmax(class_pred, dim=1)
            highest_prob_class = torch.argmax(class_probs, dim=1)
            correct += (highest_prob_class == multiclass).sum().item()
            total_samples += multiclass.size(0)

            # Compute regression MAE
            total_mae += torch.abs(reg_pred - hb_level).sum().item()

    avg_loss = total_loss / len(dataloader)
    avg_ce_loss = total_ce_loss / len(dataloader)
    avg_mse_loss = total_mse_loss / len(dataloader)
    classification_accuracy = correct / total_samples
    regression_mae = total_mae / total_samples

    return avg_loss, classification_accuracy, regression_mae, avg_ce_loss, avg_mse_loss

In [42]:
class MultiModel(nn.Module):
    def __init__(self, model_name):
        super(MultiModel, self).__init__()
        self.model_name = model_name.lower()
        self.model, num_ftrs = self._get_base_model(self.model_name)

        # Modify the classifier/head for multi-output
        if "resnet" in self.model_name:
            self.model.fc = nn.Sequential(
                nn.Dropout(p=0.2),
                nn.Linear(num_ftrs, 128),
                nn.ReLU(),
                nn.Linear(128, 5)
            )
        elif "densenet" in self.model_name or "vgg" in self.model_name:
            self.model.classifier = nn.Sequential(
                nn.Dropout(p=0.2),
                nn.Linear(num_ftrs, 128),
                nn.ReLU(),
                nn.Linear(128, 5)
            )
        elif "mobilenet" in self.model_name or "efficientnet" in self.model_name or "convnext" in self.model_name:
            self.model.classifier = nn.Sequential(
                nn.Dropout(p=0.2),
                nn.Linear(num_ftrs, 128),
                nn.ReLU(),
                nn.Linear(128, 5)
            )
        elif "vit" in self.model_name:
            self.model.head = nn.Sequential(
                nn.Dropout(p=0.2),
                nn.Linear(num_ftrs, 128),
                nn.ReLU(),
                nn.Linear(128, 5)
            )
        elif "shufflenet" in self.model_name or "regnet" in self.model_name:
            self.model.fc = nn.Sequential(
                nn.Dropout(p=0.2),
                nn.Linear(num_ftrs, 128),
                nn.ReLU(),
                nn.Linear(128, 5)
            )
        else:
            raise ValueError(f"Model {model_name} not supported")

    def _get_base_model(self, model_name):
        if model_name == "mobilenetv2":
            model = models.mobilenet_v2(pretrained=False)
            num_ftrs = model.classifier[1].in_features
        elif model_name == "resnet18":
            model = models.resnet18(pretrained=False)
            num_ftrs = model.fc.in_features
        elif model_name == "densenet121":
            model = models.densenet121(pretrained=False)
            num_ftrs = model.classifier.in_features
        elif model_name == "vgg16":
            model = models.vgg16(pretrained=False)
            num_ftrs = model.classifier[0].in_features
        elif model_name == "vit-tiny":
            model = create_model("vit_tiny_patch16_224", pretrained=False)
            num_ftrs = model.head.in_features
        # elif model_name == "convnext-tiny":
        #     model = models.convnext_tiny(pretrained=False)
        #     num_ftrs = model.classifier[1].in_features
        elif model_name == "efficientnet-b0":
            model = models.efficientnet_b0(pretrained=False)
            num_ftrs = model.classifier[-1].in_features
        elif model_name == "shufflenetv2-0.5x":
            model = models.shufflenet_v2_x0_5(pretrained=False)
            num_ftrs = model.fc.in_features
        elif model_name == "regnety-400mf":
            model = models.regnet_y_400mf(pretrained=False)
            num_ftrs = model.fc.in_features
        # elif model_name == "mnasnet0_5":
        #     model = models.mnasnet0_5(pretrained=True)
        #     num_ftrs = model.classifier[1].in_features
        # elif model_name == "ghostnet_100":
        #     model = create_model("ghostnet_100", pretrained=False)
        #     num_ftrs = model.num_features
        # elif model_name == "tinynet-a":
        #     model = create_model("tinynet_a", pretrained=False)
        #     num_ftrs = model.num_features
        else:
            raise ValueError(f"Model {model_name} not supported")
        print(f"Modified Backbone {get_model_size(model)}\n")
        return model, num_ftrs

    def forward(self, x):
        output = self.model(x)
        class_output = output[:, :4]  # First 4 values = class probabilities
        reg_output = output[:, 4]  # Last value = Hb level estimate
        return class_output, reg_output  # Return as two separate outputs

In [43]:
models_list = ["mobilenetv2", "resnet18", "densenet121", "vgg16", "vit-tiny",
               "efficientnet-b0", "shufflenetv2-0.5x", "regnety-400mf",
               #"convnext-tiny", #"mnasnet0_5", "ghostnet_100", "tinynet-a"
               ]
for arch in models_list:
    print(f"Loading model: {arch}")
    model = MultiModel(arch).to(device)
    # print(summary(model))

Loading model: mobilenetv2
Modified Backbone Model Size: 14.24 MB

Loading model: resnet18
Modified Backbone Model Size: 46.83 MB

Loading model: densenet121
Modified Backbone Model Size: 32.47 MB

Loading model: vgg16
Modified Backbone Model Size: 553.44 MB

Loading model: vit-tiny
Modified Backbone Model Size: 22.92 MB

Loading model: efficientnet-b0
Modified Backbone Model Size: 21.43 MB

Loading model: shufflenetv2-0.5x
Modified Backbone Model Size: 5.59 MB

Loading model: regnety-400mf
Modified Backbone Model Size: 17.61 MB



In [44]:
BATCH_SIZE = 32
EPOCHS = 10
FOLDS = 5

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_fn_class = torch.nn.CrossEntropyLoss()  # Multi-class classification loss
loss_fn_reg = torch.nn.MSELoss()  # Regression loss

# 5-Fold Cross Validation
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

# Directory to save the best model
weights_dir = "weights"
os.makedirs(weights_dir, exist_ok=True)

for arch in models_list:
  print(f"Training Model: {arch}")
  # Initialize model and loss functions
  model = MultiModel(arch).to(device)
  # print(f"\n{summary(model)}")
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

  best_val_acc = -float('inf')  # Track best validation accuracy
  train_metrics_df = []
  val_metrics_df = []

  # Training loop
  for epoch in range(EPOCHS):
      print(f"Epoch {epoch+1}/{EPOCHS}")
      fold = 1

      for train_idx, val_idx in kf.split(range(len(image_dataset))):  # FIX: Ensure correct splitting
          train_subset = Subset(image_dataset, train_idx)
          val_subset = Subset(image_dataset, val_idx)

          train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
          val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

          if fold == FOLDS:
              # Validation phase
              avg_val_loss, val_acc, val_mae_loss, val_ce_loss, val_mse_loss = eval(val_loader, model, loss_fn_class, loss_fn_reg)

              print(f"Validation: Fold {fold} - Total Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.4f}, CrossEntropy: {val_ce_loss:.4f}, MSE: {val_mse_loss:.4f}, MAE: {val_mae_loss:.4f}")

              # Save model with the best validation accuracy
              if val_acc > best_val_acc:
                  best_val_acc = val_acc
                  val_metrics_dict = {"Loss": avg_val_loss, "Accuracy": val_acc}
                  val_metrics_df.append(val_metrics_dict)
                  torch.save(model.state_dict(), f"{weights_dir}/model_best_accuracy_{arch}_{signature}.pth")
                  print(f"Best model saved with Accuracy: {best_val_acc:.4f}")

          else:
              # Training phase
              avg_train_loss, train_acc, train_mae_loss, train_ce_loss, train_mse_loss = train(train_loader, model, loss_fn_class, loss_fn_reg, optimizer)

              print(f"Training: Fold {fold} - Total Loss: {avg_train_loss:.4f}, Accuracy: {train_acc:.4f}, CrossEntropy: {train_ce_loss:.4f}, MSE: {train_mse_loss:.4f}, MAE: {train_mae_loss:.4f}")

              train_metrics_dict = {"Loss": avg_train_loss, "Accuracy": train_acc, "CrossEntropy": train_ce_loss, "MSE":train_mse_loss}
              train_metrics_df.append(train_metrics_dict)

          fold += 1  # Move to next fold

  # Ensure `get_model_size()` exists or remove this line
  print(f"\nFine-tuned {get_model_size(model)}\n")

Training Model: mobilenetv2
Modified Backbone Model Size: 14.24 MB

Epoch 1/10
Training: Fold 1 - Total Loss: 27.7960, Accuracy: 0.3415, CrossEntropy: 1.4209, MSE: 89.3380, MAE: 289.9852
Training: Fold 2 - Total Loss: 15.5798, Accuracy: 0.4032, CrossEntropy: 1.4482, MSE: 48.5535, MAE: 208.3845
Training: Fold 3 - Total Loss: 7.1776, Accuracy: 0.3785, CrossEntropy: 1.4314, MSE: 20.5855, MAE: 128.1004
Training: Fold 4 - Total Loss: 3.5095, Accuracy: 0.4173, CrossEntropy: 1.2931, MSE: 8.6812, MAE: 78.2320
Validation: Fold 5 - Total Loss: 6.4244, Accuracy: 0.3873, CrossEntropy: 1.2657, MSE: 18.4615, MAE: 108.8286
Best model saved with Accuracy: 0.3873
Epoch 2/10
Training: Fold 1 - Total Loss: 2.5024, Accuracy: 0.4085, CrossEntropy: 1.2434, MSE: 5.4400, MAE: 59.6949
Training: Fold 2 - Total Loss: 2.3756, Accuracy: 0.3838, CrossEntropy: 1.2377, MSE: 5.0308, MAE: 56.6373
Training: Fold 3 - Total Loss: 2.3517, Accuracy: 0.3803, CrossEntropy: 1.2533, MSE: 4.9147, MAE: 55.8373
Training: Fold 4 - 

In [45]:
for arch in models_list:
  print(f"{arch}")
  model = MultiModel(arch).to(device)
  model.load_state_dict(torch.load(f"{weights_dir}/model_best_accuracy_{arch}_{signature}.pth"))
  print(f"Fine-tuned {get_model_size(model)}")
  avg_test_loss, test_acc, test_mae_loss, test_ce_loss, test_mse_loss = eval(test_loader, model, loss_fn_class, loss_fn_reg)
  print(f"Testing: Total Loss: {avg_test_loss:.4f}, Accuracy: {test_acc:.4f}, CrossEntropy: {test_ce_loss:.4f}, MSE: {test_mse_loss:.4f}, MAE: {test_mae_loss:.4f}\n")
  %timeit avg_test_loss, test_acc, test_mae_loss, test_ce_loss, test_mse_loss = eval(test_loader, model, loss_fn_class, loss_fn_reg)
  print("")


mobilenetv2
Modified Backbone Model Size: 14.24 MB

Fine-tuned Model Size: 9.78 MB
Testing: Total Loss: 2.4636, Accuracy: 0.3873, CrossEntropy: 1.2276, MSE: 5.3478, MAE: 55.2533

431 ms ± 41.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

resnet18
Modified Backbone Model Size: 46.83 MB

Fine-tuned Model Size: 45.04 MB
Testing: Total Loss: 2.5226, Accuracy: 0.4789, CrossEntropy: 1.2259, MSE: 5.5484, MAE: 56.9497

398 ms ± 6.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

densenet121
Modified Backbone Model Size: 32.47 MB

Fine-tuned Model Size: 28.91 MB
Testing: Total Loss: 2.3085, Accuracy: 0.5634, CrossEntropy: 0.9833, MSE: 5.4008, MAE: 55.8727

724 ms ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

vgg16
Modified Backbone Model Size: 553.44 MB

Fine-tuned Model Size: 71.72 MB
Testing: Total Loss: 2.5480, Accuracy: 0.4014, CrossEntropy: 1.2369, MSE: 5.6075, MAE: 56.0298

922 ms ± 35.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

vit-tiny
Modi